In [1]:
from keras.models import  load_model
from keras.utils import np_utils
import glob
import numpy as np
import os
import sed_eval
import dcase_util

Using TensorFlow backend.


In [2]:
%run shared_functions.ipynb
nb_classes=42
from PIL import Image

In [3]:

def getStackFeatMat(scene):
    with open(scene, "rb" ) as scenesample:
        l,fv = pickle.load(scenesample)
    return fv,l

def reduceLabels(labels):
    r,c=labels.shape 
    npOfLabels=np.zeros(c)
    for j in range(0,c):
        if np.sum(labels[:,j])>0:
            npOfLabels[j]=1
    return npOfLabels

def segmentRecording(recording,labels,segmentsize,numofsegs,context):
    r,c,chanel=recording.shape   #128,1292
    if context=='asc':
        npOfLabels=np.zeros((numofsegs,10))
        npOfSegments=np.zeros((numofsegs,r,segmentsize))
        recording=recording[:,:,0]
    elif context=='sed':
        npOfLabels=np.zeros((numofsegs,32))
        npOfSegments=np.zeros((numofsegs,r,segmentsize))
        recording=recording[:,:,1]
    else:
        npOfLabels=np.zeros((numofsegs,42))
        npOfSegments=np.zeros((numofsegs,r,segmentsize,2)) #129 segments, one per 100ms, indexed 0-9
    segmentID=0
    for i in range(0,c-99,segmentsize): #start at one to remove first and last frame to have int number of frames per second
        if context in ['asc','sed']:
            npOfSegments[segmentID,:,:]=recording[:,i:i+segmentsize]
            segmentLB=np.transpose(labels[i:i+segmentsize])
        else:
            npOfSegments[segmentID,:,:,:]=recording[:,i:i+segmentsize,:]
            segmentLB=np.transpose(labels[i:i+segmentsize,:])
        lr,lc=segmentLB.shape
        newseglabels=np.zeros(lr)
        for j in range(0,lr):
            if np.sum(segmentLB[j,:])>0:
                newseglabels[j]=1
        if context=='asc':
            npOfLabels[segmentID,:]=newseglabels[0:10]
        elif context=='sed':
            npOfLabels[segmentID,:]=newseglabels[10:]
        else:
            npOfLabels[segmentID,:]=newseglabels
        segmentID=segmentID+1
    return npOfSegments, npOfLabels


def getTestDataForFold(tef_list,channels,nb_classes,segs=0,segsize=1292,context=''):
   # print('testing list is '+str(len(tef_list))+' long')
    pos=0
    if segs>1:
        if channels==2:
            X_test=np.zeros((len(tef_list)*segs,segsize,128,channels))
            Y_test=np.zeros((len(tef_list)*segs,nb_classes))
        else:
            X_test=np.zeros((len(tef_list)*segs,segsize,128))
            Y_test=np.zeros((len(tef_list)*segs,nb_classes))  
    else:   
        if channels==2:
            X_test=np.zeros((len(tef_list),1292,128,channels))
            Y_test=np.zeros((len(tef_list),1292,nb_classes)) 
        else:
            X_test=np.zeros((len(tef_list),1292,128))
            Y_test=np.zeros((len(tef_list),1292,nb_classes)) 
            
    for i,tefile in enumerate(tef_list):
        fv,l=getStackFeatMat(tefile)
        if context=='asc':
            fv=fv[:,:,0]
            l=l[:,0:10]
        elif context=='sed':
            fv=fv[:,:,1]
            l=l[:,10:]
        if segs>1:  
            fv,l=segmentRecording(fv,l,segsize,segs,context) 
            if channels==2:
                X_test[pos:pos+segs,:,:,:]=np.swapaxes(fv,1,2)
                Y_test[pos:pos+segs,:]=reduceLabels(l)
            else:
                X_test[pos:pos+segs,:,:]=np.swapaxes(fv,1,2)
                Y_test[pos:pos+segs,:]=reduceLabels(l)
        else:
            if channels==2:
                X_test[i,:,:,:]=np.swapaxes(fv,0,1)
                Y_test[i,:]=l
            else: 
                X_test[i,:,:]=np.swapaxes(fv,0,1)
                Y_test[i,:]=l
        pos=pos+segs
    return X_test, Y_test
#fold=1
#segs=1
#segsize=1292
#channels=2
#context=''
#nb_classes=42
#src_files='stacked_for_classification\\test\\'+str(fold)+'\\*2channel.pckl'
#tef_list=glob.glob(src_files)
#X_test,Y_test=getTestDataForFold(tef_list,channels,nb_classes,segs,segsize,context)
#print(X_test.shape)
#print(Y_test.shape)

In [4]:
#evaluate performance of each model
path=''
modeltype=['asc\\','sed\\','']
folders=['2D_models']

modelpathlist=[]
for fold in range(1,6):
    for m in modeltype:
        for f in folders:
            hdffile='\\f'+str(fold)+'-best_model-strongFrames.hdf5'
            route=path+m+f+hdffile
            if not os.path.isfile(route):
                print(route)
            else:
                modelpathlist.append(route)
            hdffile='\\f'+str(fold)+'-best_model-strongFramesV2.hdf5'
            route=path+m+f+hdffile
            if not os.path.isfile(route):
                print(route)
            else:
                modelpathlist.append(route)
                
print(modelpathlist)

asc\2D_models\f2-best_model-strongFramesV2.hdf5
sed\2D_models\f2-best_model-strongFramesV2.hdf5
2D_models\f2-best_model-strongFramesV2.hdf5
asc\2D_models\f3-best_model-strongFramesV2.hdf5
sed\2D_models\f3-best_model-strongFramesV2.hdf5
2D_models\f3-best_model-strongFramesV2.hdf5
asc\2D_models\f4-best_model-strongFramesV2.hdf5
sed\2D_models\f4-best_model-strongFramesV2.hdf5
2D_models\f4-best_model-strongFramesV2.hdf5
asc\2D_models\f5-best_model-strongFramesV2.hdf5
sed\2D_models\f5-best_model-strongFramesV2.hdf5
2D_models\f5-best_model-strongFramesV2.hdf5
['asc\\2D_models\\f1-best_model-strongFrames.hdf5', 'asc\\2D_models\\f1-best_model-strongFramesV2.hdf5', 'sed\\2D_models\\f1-best_model-strongFrames.hdf5', 'sed\\2D_models\\f1-best_model-strongFramesV2.hdf5', '2D_models\\f1-best_model-strongFrames.hdf5', '2D_models\\f1-best_model-strongFramesV2.hdf5', 'asc\\2D_models\\f2-best_model-strongFrames.hdf5', 'sed\\2D_models\\f2-best_model-strongFrames.hdf5', '2D_models\\f2-best_model-strongFra

In [ ]:
def getValDataForFold(foldNo,val_ID):
    n_bg,bg_labels=countBackgrounds('inputsounds\\background\\train')
    src_files='stacked_for_classification\\train\\'+str(foldNo)+'\\*'
    print(val_ID)
    vf_list=[]
    vl_list=[]
    
    while len(vf_list)==0:
        for bgl in bg_labels.keys():
            s_files=src_files+bgl+'_*2channel.pckl'
            filelist=glob.glob(s_files)
            for trainfile in filelist:  #this list is 150 files long, ie 3pitch*50 samples per 1 class
                src=trainfile.split('\\')[-1].split('_')[0]
                bgs=''.join(i for i in src if i.isdigit())
                if int(bgs) == val_ID: #then is a validation file
                     vf_list.append(trainfile)            
    print('validation list is '+str(len(vf_list))+' long')
    X_val=np.zeros((len(vf_list),1292,128,2))
    Y_val=np.zeros((len(vf_list),1292,nb_classes))
    for i,vfile in enumerate(vf_list):
        fv,l=getStackFeatMat(vfile)
        X_val[i,:,:,:]=np.swapaxes(fv,0,1)
        Y_val[i,:]=l
    return X_val, Y_val

def getValidationThreshdolds(f, v, model):
    classThresholds=np.zeros((1,32))
    classCounts=np.zeros((1,32))
    X_val, Y_val = getValDataForFold(f,v) 
    numValSamples=X_val.shape[0]  #X_val.shape=(300,1292,128)
    for ct in range(0,32):
        first=True
        for i in range(0,numValSamples):
            if np.sum(Y_val[i,:,ct]) >1: #class is present
                if first and len(X_val.shape) > 2:
                    X_cVal=np.expand_dims(X_val[i,:,:,:], axis=0)
                    Y_cVal=np.expand_dims(Y_val[i,:,:], axis=0)
                    first=False
                elif first:    
                    X_cVal=np.expand_dims(X_val[i,:,:], axis=0)
                    Y_cVal=np.expand_dims(Y_val[i,:,:], axis=0)
                    first=False
                elif len(X_val.shape) > 2:
                    X_cVal=np.concatenate((X_cVal,np.expand_dims(X_val[i,:,:,:], axis=0)),axis=0)
                    Y_cVal=np.concatenate((Y_cVal,np.expand_dims(Y_val[i,:,:], axis=0)),axis=0)
                else:
                    X_cVal=np.concatenate((X_cVal,np.expand_dims(X_val[i,:,:], axis=0)),axis=0)
                    Y_cVal=np.concatenate((Y_cVal,np.expand_dims(Y_val[i,:,:], axis=0)),axis=0)
                        
      #  print(X_cVal.shape)   
      #  print(Y_cVal.shape)
        classpredictions=model.predict(X_cVal, verbose=1, batch_size=8)
        print(classpredictions[:,:,ct].shape)
        classpredictions=classpredictions[:,:,ct]
        for sample in classpredictions:
            print(sample[100:200])

        #classpredictions[:,ct]
                
    return classThresholds

typeDict={'asc':0, 'sed':1, '':2}
validations=np.array([[5,4,1,5,5],[2,1,7,6,8],[8,8,7,7,7]])
    
v=validations[typeDict[context],fold-1]
thresholds=getValidationThreshdolds(fold, v,model)

In [8]:
#evaluate performance of each model
segsCounts={1: 1292,5: 258,10: 129,30: 43, 43: 30,100: 12,1292: 1}
   
#modelpathlist=['asc\\2D_models\\f1-best_model-strongFrames_8finish.hdf5']
for route in modelpathlist:
    print(route)
    model=load_model(route)
    channels=0
    if route[0:3] in ['asc','sed']:
        channels=1
        if route[0:3] == 'asc':
            nb_classes=10
        else:
            nb_classes=32
    else:
        channels=2
        nb_classes=42
        
    context=route[0:3]
    if context not in ['asc','sed']:
        context=''
    src_files='stacked_for_classification\\test\\'+str(fold)+'\\*2channel.pckl'
    tef_list=glob.glob(src_files)
    X_test,Y_test=getTestDataForFold(tef_list,channels,nb_classes,context=context)
     
    if 'ms' in route: #segmentation required
        segsize=int(route.split('-')[-1].split('ms')[0])
        segs=segsCounts[segsize]
        X_test,Y_test=getTestDataForFold(tef_list,channels,nb_classes,segs,segsize,context) ##
    else:
        X_test,Y_test=getTestDataForFold(tef_list,channels,nb_classes,segs=1,context=context) 
        print(Y_test.shape)
        
    if '-v3' in route:
        X_test=np.reshape(X_test,(X_test.shape[0]*X_test.shape[1],128,2))
        Y_test=np.reshape(Y_test,(Y_test.shape[0]*Y_test.shape[1],nb_classes))
    
    if len(X_test.shape)==3:
        X_test=np.expand_dims(X_test,axis=3)
    fold=route.split('-best')[0][-1:]
    predictions=np.zeros((Y_test.shape))
    print('**')
    predictions=model.predict(X_test, verbose=0, batch_size=8) 
    print(predictions.shape)
    if 'asc' == route[0:3]:
        globalthreshold=0.99
        test_scenes=hv(predictions,globalthreshold)
        asc(test_scenes,'test')
        continue
    if 'sed' == route[0:3]:
        globalthreshold=0.325
        test_events=hv(predictions,globalthreshold)
        pairfilename=sed(test_events,str(fold))
        processSEDfiles(pairfilename)
        continue
    else:
        globalthreshold=0.99
        test_scenes=hv(predictions[:,:,0:10],globalthreshold)
        asc(test_scenes,'test')
        globalthreshold=0.28
        test_events=hv(predictions[:,:,10:],globalthreshold)
        pairfilename=sed(test_events,str(fold))
        processSEDfiles(pairfilename)
    print("FIIIINESSSHED :-D")

asc\2D_models\f1-best_model-strongFrames.hdf5
asc\2D_models\f1-best_model-strongFramesV2.hdf5
sed\2D_models\f1-best_model-strongFrames.hdf5
sed\2D_models\f1-best_model-strongFramesV2.hdf5
2D_models\f1-best_model-strongFrames.hdf5
2D_models\f1-best_model-strongFramesV2.hdf5
asc\2D_models\f2-best_model-strongFrames.hdf5
(600, 1292, 10)
**
(600, 1292, 10)
*** ASC:
correct scenes: 598
 Accuracy:
0.9966666666666667
sed\2D_models\f2-best_model-strongFrames.hdf5
(600, 1292, 32)
**
(600, 1292, 32)
600
Segment based metrics
  Evaluated length                  : 14998.45 sec
  Evaluated files                   : 600 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 29.04 %
    Precision                       : 39.58 %
    Recall                          : 22.94 %
  Error rate
    Error rate (ER)                 : 0.87 
    Substitution rate               : 0.25 
    Deletion rate                   : 0.52 
    Inse


FIIIINESSSHED :-D
asc\2D_models\f3-best_model-strongFrames.hdf5
(600, 1292, 10)
**
(600, 1292, 10)
*** ASC:
correct scenes: 600
 Accuracy:
1.0
sed\2D_models\f3-best_model-strongFrames.hdf5
(600, 1292, 32)
**
(600, 1292, 32)
600
Segment based metrics
  Evaluated length                  : 15126.78 sec
  Evaluated files                   : 600 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 28.53 %
    Precision                       : 39.50 %
    Recall                          : 22.32 %
  Error rate
    Error rate (ER)                 : 0.86 
    Substitution rate               : 0.25 
    Deletion rate                   : 0.52 
    Insertion rate                  : 0.09 
  Accuracy
    Sensitivity                     : 22.32 %
    Specificity                     : 98.74 %
    Balanced accuracy               : 60.53 %
    Accuracy                        : 96.04 %

  Class-wise average metrics (macro-av


FIIIINESSSHED :-D
asc\2D_models\f4-best_model-strongFrames.hdf5
(600, 1292, 10)
**
(600, 1292, 10)
*** ASC:
correct scenes: 600
 Accuracy:
1.0
sed\2D_models\f4-best_model-strongFrames.hdf5
(600, 1292, 32)
**
(600, 1292, 32)
600
Segment based metrics
  Evaluated length                  : 15200.25 sec
  Evaluated files                   : 600 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 26.60 %
    Precision                       : 36.05 %
    Recall                          : 21.07 %
  Error rate
    Error rate (ER)                 : 0.89 
    Substitution rate               : 0.27 
    Deletion rate                   : 0.52 
    Insertion rate                  : 0.10 
  Accuracy
    Sensitivity                     : 21.07 %
    Specificity                     : 98.65 %
    Balanced accuracy               : 59.86 %
    Accuracy                        : 95.96 %

  Class-wise average metrics (macro-av


FIIIINESSSHED :-D
asc\2D_models\f5-best_model-strongFrames.hdf5
(600, 1292, 10)
**
(600, 1292, 10)
*** ASC:
correct scenes: 600
 Accuracy:
1.0
sed\2D_models\f5-best_model-strongFrames.hdf5
(600, 1292, 32)
**
(600, 1292, 32)
600
Segment based metrics
  Evaluated length                  : 15728.25 sec
  Evaluated files                   : 600 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 33.18 %
    Precision                       : 40.25 %
    Recall                          : 28.22 %
  Error rate
    Error rate (ER)                 : 0.86 
    Substitution rate               : 0.28 
    Deletion rate                   : 0.44 
    Insertion rate                  : 0.14 
  Accuracy
    Sensitivity                     : 28.22 %
    Specificity                     : 98.28 %
    Balanced accuracy               : 63.25 %
    Accuracy                        : 95.52 %

  Class-wise average metrics (macro-av


FIIIINESSSHED :-D


In [5]:
def join(test_scenes):
    asc(test_scenes,'test')
    sed(test_scenes,'test')
#ASC
#import pdb

globalthreshold=0.70

def hotvector(probs,thresh):
    s,r,c=probs.shape
   # print(s,r,c)
    hvprobs=[]
    for sample in probs:
        hv=np.zeros((sample.shape))
        for i in range(0,sample.shape[0]):
            for j in range(0,sample.shape[1]):
                if sample[i,j]>thresh:
                    hv[i,j]=1
        hvprobs.append(hv)
    return hvprobs

def hotvector_variable(probs,thresholds):
    #check number of classes matches the number of thresholds
    s,r,c=probs.shape
    print(probs.shape)
    print(thresholds.shape)
    if c==len(thresholds):
        print('yay')
    hvprobs=[]
    for sample in probs:
        hv=np.zeros((sample.shape))
        for i in range(0,sample.shape[0]):
            for j in range(0,sample.shape[1]):
                if sample[i,j]>thresholds[j]:
                    hv[i,j]=1
        hvprobs.append(hv)
    return hvprobs

predictedClass=[]

def hv(preds,globalthreshold):
    scene=[]
    hv=hotvector(preds,globalthreshold)
    #now we have hot vector for each recording,
    #for each recording get predicted scene
    for hvm in hv:
        scene.append(hvm)
    return scene

def hv_variable(preds,clThresh):
    scene=[]
    hv=hotvector_variable(preds,clThresh)
    #now we have hot vector for each recording,
    #for each recording get predicted scene
    for hvm in hv:
        scene.append(hvm)
    return scene

#test_scenes=hv(predictions)

def asc(scene,evaluate):
    bgt=[]
    acc=[]    
    n_bg, bg_labels = countBackgrounds('inputsounds\\background\\train\\')
    for bg in bg_labels:
        foo=''.join(i for i in bg if not i.isdigit())
        if foo not in bgt:
            bgt.append(foo)

    #scene is hv matrix of 1292 by 10 classes
    votes={}
    testcounter=0
    correct=0
    for idx,s in enumerate(scene):  
       # print(s.shape) #is this binarised with global threshold? yes it is.
        r,c=s.shape #this matches Y_test[idx][0:10]
        # for each class, how many 1's are there in the frames
        # highest number of 1s in frames for that class is the prediction for the recording
        for j in range(0,c): #this is 10 classes
            votes[j]=sum(s[:,j])
        #then compare to gt
        maxValue = max(votes.values())  #<-- max of values
        classes=[key for key in votes if votes[key]==maxValue]
        votes={}
        #get predicted class from gt
        for j in range(0,c):
            if Y_test[idx][:,j].any()==1:
                gt_class=j
                continue
        if gt_class in classes:
            correct=correct+1
    
    print('*** ASC:')
    print('correct scenes: ' +str(correct)+ '\n Accuracy:')
    print(correct/600) 
    return correct/600
#asc(test_scenes,'test')

In [6]:
#val_events
#test_events
import sed_eval
#%run shared_functions.ipynb

def secsFromFrame(frame):
    spf=float(30/1292)
    return float(frame*spf)

def frameFromSec(time):
    fps=float(1292/30)
    return float(time*fps)
    
def createDectectionFile(predictionsForRecording,filename,evaluate,f):
    evaluationpath='evaluation\\'+str(f)+'\\'
    savefilename=evaluationpath+filename.replace('.pckl','_'+evaluate+'-detections.txt')
    with open(savefilename,'w') as pf:
        n_fg,fgl=foregroundLabels()
        fgllist=list(fgl.keys())
        padded_predictions=onoffsets(predictionsForRecording)
        r,c=padded_predictions.shape
        waitingOffset=np.zeros((32))
        for i in range(0,r): #frames
            for j in range(0,c): #classes
                if padded_predictions[i,j]==1: #onset
                    if waitingOffset[j]==0:
                        onset=str(secsFromFrame(i)) # change for idx of recording * segment no
                        waitingOffset[j]=1
                elif padded_predictions[i,j]==-1: #offset
                    if waitingOffset[j]==1:
                        offset=str(secsFromFrame((i)))
                        waitingOffset[j]=0
                        pf.write(onset+'\t'+offset+'\t'+str(fgllist[j])+'\n')  
              
    return savefilename


def displayMatrix(mat,save=False):
    img = Image.fromarray(data, 'RGB')
    if save:
        img.save('my.png')
    img.show()
    
def onoffsets(predictionsForRecording):
    #pad predictions (time axis!)
  #  displayMatrix(predictionsForRecording)
    #r is time
    pad=np.zeros((1,32))
    paddedpredictions=np.concatenate((pad,predictionsForRecording,pad))
    differences=np.zeros((paddedpredictions.shape))
    r,c=differences.shape #r=1292, c=32, so by frame...(r)
    for i in range(0,r-1):
        differences[i,:]=paddedpredictions[i+1,:]-paddedpredictions[i,:]
    return differences

def createPairFile(singleEventHM,filename,evaluate,f):
    gtfile='syntheticSoundscenes\\test\\'+filename.split('_2channel')[0]+'.txt'
    if 'upshift' in gtfile:
        parts=gtfile.split('_upshift')
        gtfile=''.join(parts)
    elif 'downshift in gtfile':
        parts=gtfile.split('_downshift')
        gtfile=''.join(parts)
    dfname=createDectectionFile(singleEventHM,filename,evaluate,f)
    return gtfile,dfname    

def sed(test_events,f):
    pairfile='fold'+f+'_testpairs.txt'
    with open(pairfile,'w') as pf:
        for idx, s in enumerate(test_events): #- if test_events are recordings (eg of N segs), x_test is already segmented
            gtfile,detectionfile = createPairFile(s,tef_list[idx].split('\\')[-1],'test',f) 
            pf.write(gtfile+'\t'+detectionfile+'\n')
    return pairfile
segsCounts={1: 1292,5: 258,10: 129,30: 43, 43: 30,100: 12,1292: 1}

def classThresholds(model):
    ct=np.zeros((32))
    val_predictions=model.predict(X_val,verbose=0)
   # predict on validation to get an F1 score for each class
    return ct

#route='sed\\2DSeg_models\\f1best_model-10ms.hdf5'
#model=load_model(route)
#f=route.split('.')[0].split('f')[1][0]
#X_test,Y_test=getTestDataForFold(tef_list,f,nb_classes=32,segs=120,segsize=10,context='sed')
#X_test=np.expand_dims(X_test,axis=3)
#predictions=model.predict(X_test, verbose=0)
#classThresholds=np.mean(np.mean(predictions,axis=1),axis=0)
#classThresholds=classThresholds*2
#print(classThresholds.shape)
#print(predictions.shape)
#test_events=hv_variable(predictions,classThresholds) #<- needs new thresholds by class! 
#rs=sed(test_events,'1')
#print('done')

In [7]:
def processSEDfiles(pairfile):
    file_list=[]
    #pairfile='fold1testpairs.txt'
    with open(pairfile,'r') as pf:
        for line in pf.readlines():
            line=line.strip().split('\t')
            d={}
            d['reference_file']=line[0]
            d['estimated_file']=line[1]
            file_list.append(d)

    data = []
    all_data = dcase_util.containers.MetaDataContainer()
    for file_pair in file_list:
        reference_event_list = sed_eval.io.load_event_list(
            filename=file_pair['reference_file']
        )
        estimated_event_list = sed_eval.io.load_event_list(
            filename=file_pair['estimated_file']
        )

        data.append({'reference_event_list': reference_event_list,
                 'estimated_event_list': estimated_event_list})

        all_data += reference_event_list
    print(len(data))
    event_labels = all_data.unique_event_labels

    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
        event_label_list=event_labels,
        time_resolution=float(1.0)
    )

    for file_pair in data:
        segment_based_metrics.evaluate(
            reference_event_list=file_pair['reference_event_list'],
            estimated_event_list=file_pair['estimated_event_list']
        )
   
    print(segment_based_metrics)

In [ ]:
import itertools
def plot_confusion_matrix(mat, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          output = None):
    
    size = 3.5 + len(classes) * 0.5
    plt.figure(figsize=(size, size))
    plt.imshow(mat, interpolation='nearest', cmap=cmap)
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, horizontalalignment='right')
    plt.yticks(tick_marks, classes)

    fmt = '.0f' 
    thresh = mat.max() / 2.
    for i, j in itertools.product(range(mat.shape[0]), range(mat.shape[1])):
        plt.text(j, i, format(mat[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="white" if mat[i, j] > thresh else "black")

    plt.tight_layout()
    #plt.ylabel('True label')
    #plt.xlabel('Predicted label')
    
    if output is not None:
        plt.savefig(output, bbox_inches='tight')
        
#strip out non-supermarket events

scene_mat=np.random.randint(9, size=(10, 10))
classes=[bg_labels.keys()]
plot_confusion_matrix(scene_mat, classes, title='', output='scenes.pdf')
event_mat=np.random.randint(6, size=(6, 6))
classes=[fg_labels.keys()]
plot_confusion_matrix(event_mat, classes, title='', output='events.pdf')


In [ ]:
#print(predictions.shape)
#test_scenes=hv(predictions[:,:,0:10])
#asc_accuracy=asc(test_scenes,'test')
#classThresholds=np.mean(np.mean(predictions[:,:,10:],axis=1),axis=0) #placeholder thresholds
#classThresholds=classThresholds*1.5
#print(classThresholds.shape)
#test_events=hv_variable(predictions[:,:,10:],classThresholds) 
#test_events=hv_variable(predictions,classThresholds) 
#rs=sed(test_events,1)
processSEDfiles()